# Comment

This is the prototype of our future regressions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline
import time 

#to keep things in order, and to avoid to copy and paste everytime our functions if we want to use them in more than one folder,
#we can temporarily use this library. 
import sys

#in this way Python will search the implementations also in the path '../HelperFunctions'
sys.path.insert(0, '../HelperFunctions')
sys.path.insert(0, '../pre-processing/Clean_Data/')
sys.path.insert(0, '../Logit')

from proj1_helpers import *
from common_functions import *
from counters import *
from remove import *
from replace import *
from regressors import *
from CrossValidationFunctions import *
from functions_logistic import * 

In [2]:

def predict_logistic_labels(weights, data):
    """Generates class predictions given weights, and a test data matrix"""
    y_pred = np.dot(data, weights)
    y_pred=sigmoid(y_pred)
    y_pred[np.where(y_pred <= 0.5)] = 0
    y_pred[np.where(y_pred > 0.5)] = 1
    
    return y_pred
            
def convert_0_to_minus1(data):
    data[data == 0]= -1
    return data

def convert_minus1_to_0(data):
    data[data == -1]= 0
    return data



In [3]:
def penalized_logistic_regression(y, tx, w, lambda_):
    """return the loss and gradient."""
    #num_samples = y.shape[0]
    loss = calculate_logistic_loss(y, tx, w) + lambda_ * np.squeeze(w.T.dot(w))
    gradient = calculate_logistic_gradient(y, tx, w) + 2 * lambda_ * w
    return loss, gradient

def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
    """
    Do one step of gradient descent, using the penalized logistic regression.
    Return the loss and updated w.
    """
    loss, gradient = penalized_logistic_regression(y, tx, w, lambda_)
    w -= gamma * gradient
    return loss, w

In [4]:
def logistic_hyperparam_with_CV(y, tx, lambdas, gamma, degrees, max_iter):

    accuracy = np.zeros((len(lambdas), len(degrees)))
    
    for idx_lambda, lambda_ in enumerate(lambdas):
        for idx_degree, degree in enumerate(degrees):
                        
            x_augmented = build_poly(tx, degree)
            initial_w = np.ones((x_augmented.shape[1]))
            
            #regression with logistic method
            k_indices = build_k_indices(y, 4, 1)
            acc = cross_validation_with_logistic(y, x_augmented, k_indices, initial_w, gamma, lambda_, max_iter)        
            accuracy[idx_lambda, idx_degree] = acc
    
    #find the best using the accuracy
    max_acc = np.max(accuracy)
    print('max acc = ',max_acc)
    coordinates_best_parameter = np.where( accuracy == max_acc )
    print('coordinates_best_parameter: ', coordinates_best_parameter)
    best_lambda_acc = lambdas[ coordinates_best_parameter[0][0] ]
    best_degree_acc = degrees[ coordinates_best_parameter[1][0] ]

    return best_lambda_acc, best_degree_acc, max_acc

In [5]:
def cross_validation_with_logistic(y, x, k_indices,initial_w, gamma, lambda_,max_iter):
    """CV regression according to the splitting in train/test given by k_indices.
    
    The returned quantities are the average of the quantities computed in the single folds
    
    return the accuracy"""
    
    folds = k_indices.shape[0]
    accuracy = np.zeros(folds)
    w=initial_w
    
    for k in range(folds):
        
        #split the data in train/test
        idx = k_indices[k]
        yte = y[idx]
        if len( x.shape ) == 1:
            xte = x[idx]
        else:
            xte = x[idx,:]
            
        ytr = np.delete(y,idx,0)
        xtr = np.delete(x,idx,0)

        #learning by penalized graient descent (with regularized logistic)
        for iter_ in range(max_iter):
            _, w = learning_by_penalized_gradient(ytr, xtr, w, gamma, lambda_)
            
        #accuracy
        y_pred = predict_logistic_labels(w, xte)
        accuracy[k] = np.sum(y_pred == yte) / len(yte) 
        
        if k == 2:
            print('second K fold in CV')
   
    return np.mean(accuracy)



In [6]:
def removeConstantColumns(data):
    '''Remove columns which are constants from the data.
       
       Return data, idx_removed
    '''
    std = np.std(data, axis = 0)
    idx_removed = np.where(std==0)[0]
    if len(idx_removed >0 ):
        data = np.delete(data,idx_removed,axis=1)
    
    return data, idx_removed

def removeHighCorrelatedColumns(data, threshold = 0.8):
    '''Remove columns which are highly correlated.
       
       WARNING: the returned list idx_removed MUST be used in a for loop on the test data, removing features one by one
       
       Return data, idx_removed
    '''
    #initialize idx_removed
    idx_removed = []
        
    #Get first elements of the highly correlated couples
    R = np.ma.corrcoef(data.T)
    idx_HC = np.where( (R > threshold) & (R < 0.98))[0] 

    while(idx_HC.shape[0] > 0):
        
        idx_to_remove = idx_HC.max()
        
        data = np.delete(data, idx_to_remove, axis=1)
        idx_removed.append(idx_to_remove)
        
        #compute the correlation coefficients of the reduced dataset
        R = np.ma.corrcoef(data.T)
        idx_HC = np.where( (R > threshold) & (R < 0.98))[0] 
        
    
    return data, idx_removed



# Load Data And Basic Preprocessing

In [7]:
yb, input_data, ids = load_csv_data("../data/train.csv", sub_sample=False)
_, test_data, ids_test = load_csv_data("../data/test.csv", sub_sample=False)

#this will surely be deleted, in this way we are sure that original_data is the original version of the data and we don't have
#to load them again
from copy import deepcopy
originalData = deepcopy(input_data)
originalY = deepcopy(yb)
originalTest = deepcopy(test_data)


# Step 0

In [8]:
#basic step
input_data = deepcopy(originalData)
input_data = replaceWithZero(input_data,-999,idxCols)

# Jet division, removing constant columns, standardization

In [9]:
# x0, x1, x2
idx0 = np.where(input_data[:,22]==0)
idx1 = np.where(input_data[:,22]==1)
idx2 = np.where(input_data[:,22]>=2)

x0 = input_data[idx0] 
x1 = input_data[idx1] 
x2 = input_data[idx2] 

y0 = yb[idx0]
y1 = yb[idx1]
y2 = yb[idx2]

x0, idx_constants_removed0 = removeConstantColumns(x0)
x1, idx_constants_removed1 = removeConstantColumns(x1)

x0, mean_train0, std_train0 = standardize ( x0 )
x1, mean_train1, std_train1 = standardize ( x1 )
x2, mean_train2, std_train2 = standardize ( x2 )

In [10]:
y0 = convert_minus1_to_0(y0)
y1 = convert_minus1_to_0(y1)
y2 = convert_minus1_to_0(y2)

# Remove HC columns

Only if HC_flag = True

In [11]:
HC_flag = True

if(HC_flag):
    threshold = 0.8

    x0, idx_HC_removed0 = removeHighCorrelatedColumns(x0, threshold)
    x1, idx_HC_removed1 = removeHighCorrelatedColumns(x1, threshold)
    x2, idx_HC_removed2 = removeHighCorrelatedColumns(x2, threshold)
else:
    idx_HC_removed0 = []
    idx_HC_removed0 = []
    idx_HC_removed0 = []

# Regression....

In [12]:
max_iter=500
gamma=1e-5

w_initial_x0 = np.ones((x0.shape[1]))
w_initial_x1 = np.ones((x1.shape[1]))
w_initial_x2 = np.ones((x2.shape[1]))

for i in range(max_iter):
    _, w_x0 = learning_by_gradient_descent(y0, x0, w_initial_x0, gamma)
    _, w_x1 = learning_by_gradient_descent(y1, x1, w_initial_x1, gamma)
    _, w_x2 = learning_by_gradient_descent(y2, x2, w_initial_x2, gamma)

In [ ]:
# lambdas0 = np.linspace(0.0001,0.01,15) #for the first subset the lambda is around 0.001
# lambdas1 = np.linspace(0.00001,0.001,15) #for the second subset the lambda is around 0.0001
# lambdas2 = np.linspace(0.00001,0.001,15) #for the third subset the lambda is around 0.0001

# degrees = np.arange(7,17) #the best degree was high for all the models


# best_lambda_loss0, best_degree_loss0, best_w_loss0, best_lambda_acc0, best_degree_acc0, best_w_acc0, loss_tr0, loss_te0, accuracy0 = \
# grid_search_hyperparam_with_CV(y0, x0, lambdas0, degrees)

# best_lambda_loss1, best_degree_loss1, best_w_loss1, best_lambda_acc1, best_degree_acc1, best_w_acc1, loss_tr1, loss_te1, accuracy1 = \
# grid_search_hyperparam_with_CV(y1, x1, lambdas1, degrees)

# best_lambda_loss2, best_degree_loss2, best_w_loss2, best_lambda_acc2, best_degree_acc2, best_w_acc2, loss_tr2, loss_te2, accuracy2 = \
# grid_search_hyperparam_with_CV(y2, x2, lambdas2, degrees)

# print('LOSS')
# print(f'Model with 0 jets: lambda = {best_lambda_loss0}, degree = {best_degree_loss0}, loss = {np.min(loss_te0)}')
# print(f'Model with 1 jets: lambda = {best_lambda_loss1}, degree = {best_degree_loss1}, loss = {np.min(loss_te1)}')
# print(f'Model with more than 1 jets: lambda = {best_lambda_loss2}, degree = {best_degree_loss2}, loss = {np.min(loss_te2)}')

# print('\n\nACCURACY')
# print(f'Model with 0 jets: lambda = {best_lambda_acc0}, degree = {best_degree_acc0}, acc = {np.max(accuracy0)}')
# print(f'Model with 1 jets: lambda = {best_lambda_acc1}, degree = {best_degree_acc1}, acc = {np.max(accuracy1)}')
# print(f'Model with more than 1 jets: lambda = {best_lambda_acc2}, degree = {best_degree_acc2}, acc = {np.max(accuracy2)}')


# N0 = x0.shape[0]
# N1 = x1.shape[0]
# N2 = x2.shape[0]

# TOTAccuracy = ( N0*np.max(accuracy0) + N1*np.max(accuracy1) + N2*np.max(accuracy2) ) / ( N0 + N1 + N2 )
# print(f'\n\nOur test set reached an accuracy of: acc = {TOTAccuracy}')

#### Only for regLogit

In [ ]:
#Only for regularized Logit
gamma=1e-5

x0_augmented = build_poly(x0, best_degree_acc_0)
x1_augmented = build_poly(x1, best_degree_acc_1)
x2_augmented = build_poly(x2, best_degree_acc_2)

initial_w_x0 = np.ones((x0_augmented.shape[1]))
initial_w_x1 = np.ones((x1_augmented.shape[1]))
initial_w_x2 = np.ones((x2_augmented.shape[1]))

_, best_w_0 = learning_by_penalized_gradient(y0, x0_augmented, initial_w_x0, gamma, best_lambda_acc_0)
_, best_w_1 = learning_by_penalized_gradient(y1, x1_augmented, initial_w_x1, gamma, best_lambda_acc_1)
_, best_w_2 = learning_by_penalized_gradient(y2, x2_augmented, initial_w_x2, gamma, best_lambda_acc_2)

# Submission: import and basic steps

In [13]:
test_data = deepcopy(originalTest)
num_tests = test_data.shape[0]

numInvalidValues=countInvalid(test_data, -999)
idxCols = np.where(numInvalidValues>0)[0]
input_data = replaceWithZero(test_data,-999,idxCols)



# Jet division, removing constant/HC columns, standardization

In [14]:
# x0, x1, x2
idx0 = np.where(test_data[:,22]==0)
idx1 = np.where(test_data[:,22]==1)
idx2 = np.where(test_data[:,22]>=2)

test_x0 = test_data[idx0] 
test_x1 = test_data[idx1] 
test_x2 = test_data[idx2] 

test_x0 = np.delete(test_x0, idx_constants_removed0, axis=1)
test_x1 = np.delete(test_x1, idx_constants_removed1, axis=1)

test_x0,_,_ = standardize ( test_x0, mean_train0, std_train0 )
test_x1,_,_ = standardize ( test_x1, mean_train1, std_train1 )
test_x2,_,_ = standardize ( test_x2, mean_train2, std_train2 )

if(HC_flag):
    for i in idx_HC_removed0:
        test_x0 = np.delete(test_x0,i,axis=1)
    for i in idx_HC_removed1:
        test_x1 = np.delete(test_x1,i,axis=1)
    for i in idx_HC_removed2:
        test_x2 = np.delete(test_x2,i,axis=1)

# Repeat regression stuff and predict

#### Only for regLogit

In [ ]:
#retraining on the whole dataset using best hyperparameters
gamma = 1e-5
max_iter = 100

x0_augmented = build_poly(x0, best_degree_acc_0)
x1_augmented = build_poly(x1, best_degree_acc_1)
x2_augmented = build_poly(x2, best_degree_acc_2)

initial_w_x0 = np.ones((x0_augmented.shape[1]))
initial_w_x1 = np.ones((x1_augmented.shape[1]))
initial_w_x2 = np.ones((x2_augmented.shape[1]))

for iter_ in range(max_iter):

    _, best_w_0 = learning_by_penalized_gradient(y0, x0_augmented, initial_w_x0, gamma, best_lambda_acc_0)
    _, best_w_1 = learning_by_penalized_gradient(y1, x1_augmented, initial_w_x1, gamma, best_lambda_acc_1)
    _, best_w_2 = learning_by_penalized_gradient(y2, x2_augmented, initial_w_x2, gamma, best_lambda_acc_2)

#design of the test set for our model
test_x0_augmented = build_poly(test_x0,best_degree_acc_0)
test_x1_augmented = build_poly(test_x1,best_degree_acc_1)
test_x2_augmented = build_poly(test_x2,best_degree_acc_2)


y_pred0 = predict_logistic_labels(best_w_0,test_x0_augmented)
y_pred1 = predict_logistic_labels(best_w_1,test_x1_augmented)
y_pred2 = predict_logistic_labels(best_w_2,test_x2_augmented)


y_pred = np.ones(num_tests)
y_pred[idx0] = y_pred0
y_pred[idx1] = y_pred1
y_pred[idx2] = y_pred2

y_pred = convert_0_to_minus1(y_pred)

#### prediction for Logit 

In [16]:
y_pred0 = predict_logistic_labels(w_x0,test_x0)
y_pred1 = predict_logistic_labels(w_x1,test_x1)
y_pred2 = predict_logistic_labels(w_x2,test_x2)


y_pred = np.ones(num_tests)
y_pred[idx0] = y_pred0
y_pred[idx1] = y_pred1
y_pred[idx2] = y_pred2

y_pred = convert_0_to_minus1(y_pred)

In [17]:
create_csv_submission(ids_test, y_pred, 'Logit_RemovConstColumns_RemovHCcolumns_Normal_WithZero_JetDiv_.csv')